<a href="https://colab.research.google.com/github/gustavo-ifusp/prefeitura_smit_sp360/blob/main/016%20-%20saude_dados_remedios_versao_consumo_medio_mensal_e_controle_de_estoque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import zipfile
from datetime import date, time, datetime, timedelta
from dateutil.relativedelta import relativedelta

#**Saúde - Dados dos Remédios**
**Dados de origem:**<br>
\\\SMITGBC031\situacao_medicamentos
<br>
**Arquivos backup para teste**<br>
https://drive.google.com/drive/folders/1p1ZnLh8SUBNoygG2m1AwX3IGQ_aWstOP?usp=sharing
<br><br>
**Objetivo:**<br>
Organizar os dados por região e data, deixando no esquema star schema.<br><br>
Responsável: Gustavo de Andrade <br> gustavo.andrade@prefeitura.sp.gov.br


In [ ]:
dados = pd.read_csv("gss_posicao_estoque_20211020.csv", sep = ';', header = 0, encoding='ISO-8859-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dados

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,579
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,506
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,101
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,500
...,...,...,...,...,...,...,...,...,...,...,...,...
484735,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40
484736,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100
484737,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4
484738,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1


In [ ]:
dados['TIPO UNIDADE MEDIDA'].unique()

array(['UN ', 'FR ', 'AMP', 'FAM', 'ENV', 'CP ', 'BNG', 'M  ', 'PC ',
       'RL ', 'KIT', 'PCT', 'PAR', 'TST', 'LT ', 'CJ ', 'TB ', 'EST',
       'CAP', 'L  ', 'CX ', 'GL ', 'FL ', 'PT ', 'G  ', 'BOB', 'SRG',
       'ML ', 'DRG', 'M3 ', 'RM ', 'BLC', 'BOL', 'SCH', 'KG ', 'CPD',
       '   ', 'CT ', 'SER'], dtype=object)

In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484740 entries, 0 to 484739
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SUPERVISAO                483001 non-null  object 
 1   CNES ESTABELECIMENTO      483232 non-null  float64
 2   SUPRI ESTABELECIMENTO     483477 non-null  float64
 3   ESTABELECIMENTO           484740 non-null  object 
 4   CODIGO SUPRI ITEM         484740 non-null  object 
 5   DESCRICAO ITEM            484740 non-null  object 
 6   TIPO UNIDADE MEDIDA       484740 non-null  object 
 7   CODIGO ALMOXARIFADO ITEM  484740 non-null  int64  
 8   STATUS                    484740 non-null  object 
 9   BLOQUEIO STS              484740 non-null  object 
 10  CMM                       484554 non-null  float64
 11  ESTOQUE                   484740 non-null  int64  
dtypes: float64(3), int64(2), object(7)
memory usage: 44.4+ MB


In [ ]:
dados['SUPERVISAO'].unique()

array(['SUPERVISÃO TÉCNICA DE SAÚDE SÉ',
       'SUPERVISAO TECNICA DE SAUDE SANTA CECILIA',
       'SUPERVISÃO TÉCNICA DE SAÚDE ARICANDUVA/FORMOSA/CARRÃO/MOOCA',
       'SUPERVISÃO TÉCNICA DE SAÚDE VILA PRUDENTE/SAPOPEMBA',
       'SUPERVISÃO TÉCNICA DE SAÚDE VILA MARIANA/JABAQUARA',
       'SUPERVISÃO TÉCNICA DE SAÚDE VILA MARIA/VILA GUILHERME',
       'SUPERVISÃO TÉCNICA DE SAÚDE PENHA',
       'SUPERVISÃO TÉCNICA DE SAÚDE ITAQUERA',
       'SUPERVISAO TECNICA DE SAUDE ERMELINO MATARAZZO',
       'SUPERVISÃO TÉCNICA DE SAÚDE ITAIM PAULISTA',
       'SUPERVISÃO TÉCNICA DE SAÚDE IPIRANGA',
       'SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ',
       'SUPERVISAO TECNICA DE SAUDE PERUS', nan,
       'SUPERVISÃO TÉCNICA DE SAÚDE FREGUESIA DO Ó/BRASILÂNDIA',
       'SUPERVISÃO TÉCNICA DE SAÚDE CASA VERDE/CACHOEIRINHA',
       'SUPERVISÃO TÉCNICA DE SAÚDE SANTANA/JAÇANA/TREMEMBE/TUCURUVI',
       'SUPERVISÃO TÉCNICA DE SAÚDE CAPELA DO SOCORRO',
       'SUPERVISÃO TÉCNICA DE SAÚDE MBOI MIRIM',
    

In [ ]:
remedios = pd.DataFrame()
remedios['remedios'] = Counter(dados['DESCRICAO ITEM']).keys()
remedios['repete'] = Counter(dados['DESCRICAO ITEM']).values()
remedios

,remedios,repete
0,PARACETAMOL 500 MG COMPRIMIDO,613
1,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,695
2,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,652
3,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,692
4,DIPIRONA SODICA 500 MG COMPRIMIDO,633
...,...,...
3226,SAMU - DETERGENTE DESINCROSTANTE ORGANICO PARA...,1
3227,SACO DE ALGODAO CRU ALVEJADO - 71 X 41 CM - BR...,1
3228,"DUCHA, HIGIENICA, INTIMA, USO UNICO",1
3229,"CANULA, TRAQUEOSTOMIA, S/ BALAO, DESCARTAVEL, ...",1


#**Começo da modelagem**
Até agora foi para entender o dataframe!

In [ ]:
dados

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,579
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,506
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,101
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,500
...,...,...,...,...,...,...,...,...,...,...,...,...
484735,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40
484736,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100
484737,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4
484738,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1


In [ ]:
estoque_regiao = dados.groupby(['SUPERVISAO','DESCRICAO ITEM'])['ESTOQUE'].sum()

In [ ]:
estoque_regiao

SUPERVISAO                              DESCRICAO ITEM                                                                                      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG                                33400
                                        BENZILPENICILINA PROCAINA 300.000 UI + BENZILPENICILINA POTASSICA 100.000 UI SUSPENSAO INJETAVEL FAM        5
                                        CEFALEXINA 50 MG/ML SUSPENSAO ORAL FRASCO 60 ML                                                           492
                                        CIMETIDINE 200 MG (TAGAMET, ULCIMET)                                                                     2220
                                        DIAZEPAN EM COMPRIMIDOS SULCADOS COM 10 MG                                                               2960
                                                                                                             

In [ ]:
estoque_estabelecimento = dados.groupby(['ESTABELECIMENTO','DESCRICAO ITEM'])['ESTOQUE'].sum()

In [ ]:
estoque_estabelecimento

ESTABELECIMENTO       DESCRICAO ITEM                                                    
AE ALTO DA BOA VISTA  ABAIXADOR, LINGUA, MADEIRA, 100 UNIDADES                                4
                      ACIDO ACETICO 20 MG/ML (2%) SOLUCAO FRASCO 1 LITRO                      1
                      ACIDO ACETICO 50 MG/ML (5%) SOLUCAO FRASCO 1 LITRO                      1
                      ACIDO ACETILSALICILICO 100 MG COMPRIMIDO                               30
                      ACIDO NITRICO FUMEGANTE 660 MG/ML (66%) SOLUCAO FRASCO 10 ML            0
                                                                                           ... 
URSI TUCURUVI         TERBUTALINA 0,5 MG/ML SOLUCAO INJETAVEL AMP. 1 ML                       4
                      TIRA, REAGENTE, P/ DETERMINACAO DE GLICOSE, SANGUE, USO HOSPITALAR      0
                      TOUCA, CIRURGICA, DESCARTAVEL                                           0
                      TOUCA, CIRURGICA, DESCART

In [ ]:
estoque_estabelecimento_cnes = dados.groupby(['CNES ESTABELECIMENTO','DESCRICAO ITEM'])['ESTOQUE'].sum()

In [ ]:
estoque_estabelecimento_cnes

CNES ESTABELECIMENTO  DESCRICAO ITEM                                                                                           
85074.0               AMOXICILINA 50 MG/ML PO PARA SUSPENSAO ORAL FRASCO 150 ML                                                        0
                      AMOXICILINA 500 MG COMPRIMIDO                                                                                24000
                      AZITROMICINA 40 MG/ML SUSPENSAO ORAL FRASCO 15 ML                                                              725
                      AZITROMICINA 500 MG COMPRIMIDO                                                                                6000
                      CEFALEXINA 50 MG/ML SUSPENSAO ORAL FRASCO 60 ML                                                               1000
                                                                                                                                   ...  
9999977.0             TUBO P/ COLETA DE SANGUE A V

#**Adquirindo como teste o backup de arquivos da pasta no Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Esse codigo vai extrar todos os arquivos em uma pasta e salvar em outra:**

In [ ]:
arquivos = glob.glob('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/*.zip') #não precisa mais carregar novamente
arquivos

['/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211021.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211022.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211023.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211024.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211025.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211026.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211027.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211028.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211029.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211030.zip',


In [ ]:
n = 0
while n < len(arquivos):
    zip = zipfile.ZipFile(arquivos[n])
    zip.extractall('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraido/')
    n += 1

#**Agora o objetivo é ler cada arquivo e criar uma coluna com data que está no nome, depois concatenar todos os arquivos de forma automatica**

In [ ]:
extraidos = glob.glob('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/novembro/*.csv')
extraidos

['/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/novembro/gss_posicao_estoque_20211101.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/novembro/gss_posicao_estoque_20211102.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/novembro/gss_posicao_estoque_20211103.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/novembro/gss_posicao_estoque_20211104.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/novembro/gss_posicao_estoque_20211105.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/novembro/gss_posicao_estoque_20211106.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/novembro/gss_posicao_estoque_20211107.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/novembro/gss_posicao_estoque_20211108.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situa

In [ ]:
all_data = pd.DataFrame() 
n = 0
for f in extraidos:
    df1 = pd.read_csv(f, sep = ';', header = 0, encoding='ISO-8859-1')
    data_n = extraidos[n].split('/')
    data_n = data_n[9].split('_')
    data_n = data_n[3].split('.')
    data_n = int(data_n[0])
    data_n_ano = data_n // 10000
    data_n_mes = (data_n % 10000)//100
    data_n_dia = data_n % 10000
    data_n_dia = data_n_dia % 100
    df1['data']  = date(year = data_n_ano, month = data_n_mes, day = data_n_dia)
    all_data = all_data.append(df1)
    n += 1
all_data.sample(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
60086,SUPERVISÃO TÉCNICA DE SAÚDE ITAIM PAULISTA,2787350.0,8.426301e+09,AMA/UBS INTEGRADA JARDIM DAS OLIVEIRAS,1106500200113750,"FIO CIRURGICO, NYLON 4-0, NAO ABSORV., 75 CM, ...",ENV,8410200074,DESBLOQUEADO,NAO,5.0,0,2021-11-10
174920,SUPERVISÃO TÉCNICA DE SAÚDE ARICANDUVA/FORMOSA...,2788942.0,8.425301e+09,AMA/UBS VILA GUARANI,1106400904700409,"LEVONORGESTREL 0,75 MG COMPRIMIDO",UN,8410200066,DESBLOQUEADO,NAO,2.0,2,2021-11-13


In [ ]:
len(all_data)

8270953

In [ ]:
remedios_demanda_regiao = pd.DataFrame()
remedios_demanda_estabelecimento = pd.DataFrame()

#**Janeiro**

In [ ]:
all_data_janeiro = pd.DataFrame()
all_data_janeiro = all_data.reset_index()
all_data_janeiro

,index,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,87,2021-01-01
1,1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,30,2021-01-01
2,2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,4,2021-01-01
3,3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,15,2021-01-01
4,4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,496,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13783700,445622,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106400101100015,CLORPROMAZINA CLORIDRATO 5 MG/ML SOLUCAO INJET...,AMP,8410200104,DESBLOQUEADO,NAO,3.0,3,2021-01-31
13783701,445623,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106400101100139,HALOPERIDOL 5 MG/ML SOLUCAO INJETAVEL AMP. 1 ML,AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-01-31
13783702,445624,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106400101100147,"HALOPERIDOL DECANOATO 70,52 MG/ML (ESQUIVALENT...",AMP,8410200104,DESBLOQUEADO,NAO,15.0,17,2021-01-31
13783703,445625,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401005200226,DOXICICLINA CLORIDRATO 100 MG COMPRIMIDO,UN,8410200104,DESBLOQUEADO,NAO,1.0,20,2021-01-31


In [ ]:
len(all_data_janeiro)

13783705

In [ ]:
all_data_janeiro.to_csv(r'all_data_janeiro', index = False)

In [ ]:
all_data_janeiro["estoque"] = ""
all_data_janeiro["cmm_media"] = ""

In [ ]:
estoque_regiao_jan_estoque = all_data_janeiro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_jan_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,33400
...,...,...,...,...
1236132,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-27,0
1236133,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-28,0
1236134,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-29,0
1236135,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-30,0


In [ ]:
estoque_regiao_jan_cmm = all_data_janeiro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_jan_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,0.0
...,...,...,...,...
1236132,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-27,106848.0
1236133,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-28,106848.0
1236134,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-29,106848.0
1236135,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-30,106848.0


In [ ]:
estoque_regiao_jan = pd.merge(estoque_regiao_jan_estoque, estoque_regiao_jan_cmm, how = 'inner')

In [ ]:
estoque_regiao_jan = ""

In [ ]:
estoque_regiao_jan = estoque_regiao_jan.replace(np.nan, 0)
estoque_regiao_jan

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,33400,0.0
...,...,...,...,...,...
1236132,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-27,0,106848.0
1236133,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-28,0,106848.0
1236134,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-29,0,106848.0
1236135,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-30,0,106848.0


In [ ]:
estoque_regiao_jan

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,33400,0.0
...,...,...,...,...,...
1236132,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-27,0,106848.0
1236133,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-28,0,106848.0
1236134,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-29,0,106848.0
1236135,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-30,0,106848.0


Aqui começa o esquema estrela de modelagem para o mês de janeiro executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_jan): 
  id_supervisao = estoque_regiao_jan['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_jan['SUPERVISAO']= lista
estoque_regiao_jan = estoque_regiao_jan.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_jan

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,33400,0.0
...,...,...,...,...,...
1236132,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-27,0,106848.0
1236133,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-28,0,106848.0
1236134,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-29,0,106848.0
1236135,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-30,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_jan)): 
  id_descricao_item = estoque_regiao_jan['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_jan['DESCRICAO ITEM'] = lista
estoque_regiao_jan = estoque_regiao_jan.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_jan)

In [ ]:
remedios_demanda_regiao

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-01-01,33400,0.0
1,1,61,2021-01-02,33400,0.0
2,1,61,2021-01-03,33400,0.0
3,1,61,2021-01-04,33400,0.0
4,1,61,2021-01-05,33400,0.0
...,...,...,...,...,...
1236132,32,1298,2021-01-27,0,106848.0
1236133,32,1298,2021-01-28,0,106848.0
1236134,32,1298,2021-01-29,0,106848.0
1236135,32,1298,2021-01-30,0,106848.0


In [ ]:
estoque_regiao_jan.to_csv(r'estoque_regiao_jan.csv')

In [ ]:
estoque_regiao_jan

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-01-01,33400,0.0
1,1,61,2021-01-02,33400,0.0
2,1,61,2021-01-03,33400,0.0
3,1,61,2021-01-04,33400,0.0
4,1,61,2021-01-05,33400,0.0
...,...,...,...,...,...
1236132,32,1298,2021-01-27,0,106848.0
1236133,32,1298,2021-01-28,0,106848.0
1236134,32,1298,2021-01-29,0,106848.0
1236135,32,1298,2021-01-30,0,106848.0


#**Fevereiro**

In [ ]:
all_data_fevereiro = pd.DataFrame()
all_data_fevereiro = all_data
all_data_fevereiro

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,580,2021-02-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,490,2021-02-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,2,2021-02-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,13,2021-02-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,251,2021-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
446823,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,50,2021-02-28
446824,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-02-28
446825,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-02-28
446826,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-02-28


In [ ]:
all_data_fevereiro.to_csv(r'all_data_fevereiro', index = False)

In [ ]:
len(all_data_fevereiro)

12509993

In [ ]:
all_data_fevereiro["estoque"] = ""
all_data_fevereiro["cmm_media"] = ""

In [ ]:
estoque_regiao_fev_estoque = all_data_fevereiro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_fev_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-05,33400
...,...,...,...,...
1117159,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-24,0
1117160,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-25,0
1117161,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-26,0
1117162,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-27,0


In [ ]:
estoque_regiao_fev_cmm = all_data_fevereiro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_fev_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-05,0.0
...,...,...,...,...
1117159,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-24,106848.0
1117160,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-25,106848.0
1117161,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-26,106848.0
1117162,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-27,106848.0


In [ ]:
estoque_regiao_fev = pd.merge(estoque_regiao_fev_estoque, estoque_regiao_fev_cmm, how = 'inner')

In [ ]:
estoque_regiao_fev = estoque_regiao_fev.replace(np.nan, 0)
estoque_regiao_fev

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-05,33400,0.0
...,...,...,...,...,...
1117159,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-24,0,106848.0
1117160,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-25,0,106848.0
1117161,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-26,0,106848.0
1117162,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-27,0,106848.0



Aqui começa o esquema estrela de modelagem para o mês de janeiro executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_fev): 
  id_supervisao = estoque_regiao_fev['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_fev['SUPERVISAO']= lista
estoque_regiao_fev = estoque_regiao_fev.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_fev

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-05,33400,0.0
...,...,...,...,...,...
1117159,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-24,0,106848.0
1117160,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-25,0,106848.0
1117161,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-26,0,106848.0
1117162,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-27,0,106848.0


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_fev)): 
  id_descricao_item = estoque_regiao_fev['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_fev['DESCRICAO ITEM'] = lista
estoque_regiao_fev = estoque_regiao_fev.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_fev)

In [ ]:
estoque_regiao_fev.to_csv(r'estoque_regiao_fev.csv')

In [ ]:
estoque_regiao_fev

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-02-01,33400,0.0
1,1,61,2021-02-02,33400,0.0
2,1,61,2021-02-03,33400,0.0
3,1,61,2021-02-04,33400,0.0
4,1,61,2021-02-05,33400,0.0
...,...,...,...,...,...
1117159,32,1298,2021-02-24,0,106848.0
1117160,32,1298,2021-02-25,0,106848.0
1117161,32,1298,2021-02-26,0,106848.0
1117162,32,1298,2021-02-27,0,106848.0


#**Março**

In [ ]:
all_data_marco = pd.DataFrame()
all_data_marco = all_data
all_data_marco

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,580,2021-03-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,480,2021-03-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,2,2021-03-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,12,2021-03-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,136,2021-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
448291,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,50,2021-03-31
448292,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-03-31
448293,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-03-31
448294,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-03-31


In [ ]:
all_data_marco.to_csv(r'all_data_marco', index = False)

In [ ]:
len(all_data_marco)

13876929

In [ ]:
all_data_marco["estoque"] = ""
all_data_marco["cmm_media"] = ""

In [ ]:
estoque_regiao_mar_estoque = all_data_marco.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_mar_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-05,33400
...,...,...,...,...
1238192,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-27,0
1238193,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-28,0
1238194,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-29,0
1238195,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-30,0


In [ ]:
estoque_regiao_mar_cmm = all_data_marco.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_mar_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-05,0.0
...,...,...,...,...
1238192,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-27,106848.0
1238193,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-28,106848.0
1238194,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-29,106848.0
1238195,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-30,106848.0


In [ ]:
estoque_regiao_mar = pd.merge(estoque_regiao_mar_estoque, estoque_regiao_mar_cmm, how = 'inner')

In [ ]:
estoque_regiao_mar = estoque_regiao_mar.replace(np.nan, 0)
estoque_regiao_mar

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-05,33400,0.0
...,...,...,...,...,...
1238192,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-27,0,106848.0
1238193,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-28,0,106848.0
1238194,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-29,0,106848.0
1238195,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-30,0,106848.0



Aqui começa o esquema estrela de modelagem para o mês de janeiro executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_mar): 
  id_supervisao = estoque_regiao_mar['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_mar['SUPERVISAO'] = lista
estoque_regiao_mar = estoque_regiao_mar.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_mar

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-05,33400,0.0
...,...,...,...,...,...
1238192,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-27,0,106848.0
1238193,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-28,0,106848.0
1238194,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-29,0,106848.0
1238195,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-30,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_mar)): 
  id_descricao_item = estoque_regiao_mar['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_mar['DESCRICAO ITEM'] = lista
estoque_regiao_mar = estoque_regiao_mar.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_mar)

In [ ]:
estoque_regiao_mar.to_csv(r'estoque_regiao_mar.csv')

In [ ]:
estoque_regiao_mar

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-03-01,33400,0.0
1,1,61,2021-03-02,33400,0.0
2,1,61,2021-03-03,33400,0.0
3,1,61,2021-03-04,33400,0.0
4,1,61,2021-03-05,33400,0.0
...,...,...,...,...,...
1238192,32,1298,2021-03-27,0,106848.0
1238193,32,1298,2021-03-28,0,106848.0
1238194,32,1298,2021-03-29,0,106848.0
1238195,32,1298,2021-03-30,0,106848.0


#**Abril**

In [ ]:
all_data_abril = pd.DataFrame()
all_data_abril = all_data
all_data_abril

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,553,2021-04-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,470,2021-04-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,9,2021-04-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,12,2021-04-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,502,2021-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
449133,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,50,2021-04-30
449134,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-04-30
449135,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-04-30
449136,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-04-30


In [ ]:
all_data_abril.to_csv(r'all_data_abril.csv')

In [ ]:
len(all_data_abril)

13463250

In [ ]:
all_data_abril["estoque"] = ""
all_data_abril["cmm_media"] = ""

In [ ]:
estoque_regiao_abr_estoque = all_data_abril.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_abr_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-05,33400
...,...,...,...,...
1201042,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-26,0
1201043,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-27,0
1201044,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-28,0
1201045,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-29,0


In [ ]:
estoque_regiao_abr_cmm = all_data_abril.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_abr_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-05,0.0
...,...,...,...,...
1201042,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-26,106848.0
1201043,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-27,106848.0
1201044,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-28,106848.0
1201045,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-29,106848.0


In [ ]:
estoque_regiao_abr = pd.merge(estoque_regiao_abr_estoque, estoque_regiao_abr_cmm, how = 'inner')

In [ ]:
estoque_regiao_abr = estoque_regiao_abr.replace(np.nan, 0)
estoque_regiao_abr

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-05,33400,0.0
...,...,...,...,...,...
1201042,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-26,0,106848.0
1201043,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-27,0,106848.0
1201044,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-28,0,106848.0
1201045,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-29,0,106848.0



Aqui começa o esquema estrela de modelagem para o mês de janeiro executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_abr): 
  id_supervisao = estoque_regiao_abr['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_abr['SUPERVISAO']= lista
estoque_regiao_abr = estoque_regiao_abr.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_abr

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-05,33400,0.0
...,...,...,...,...,...
1201042,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-26,0,106848.0
1201043,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-27,0,106848.0
1201044,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-28,0,106848.0
1201045,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-29,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_abr)): 
  id_descricao_item = estoque_regiao_abr['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_abr['DESCRICAO ITEM'] = lista
estoque_regiao_abr = estoque_regiao_abr.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
estoque_regiao_abr.to_csv(r'estoque_regiao_abr.csv')

In [ ]:
estoque_regiao_abr

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-04-01,33400,0.0
1,1,61,2021-04-02,33400,0.0
2,1,61,2021-04-03,33400,0.0
3,1,61,2021-04-04,33400,0.0
4,1,61,2021-04-05,33400,0.0
...,...,...,...,...,...
1201042,32,1298,2021-04-26,0,106848.0
1201043,32,1298,2021-04-27,0,106848.0
1201044,32,1298,2021-04-28,0,106848.0
1201045,32,1298,2021-04-29,0,106848.0


#**Maio**

In [ ]:
all_data_maio = pd.DataFrame()
all_data_maio = all_data
all_data_maio

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,587,2021-05-22
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,470,2021-05-22
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,10,2021-05-22
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,13,2021-05-22
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,252,2021-05-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
449200,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,50,2021-05-21
449201,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-05-21
449202,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-05-21
449203,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-05-21


In [ ]:
len(all_data_maio)

13925016

In [ ]:
all_data_maio["estoque"] = ""
all_data_maio["cmm_media"] = ""

In [ ]:
estoque_regiao_mai_estoque = all_data_maio.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_mai_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-05,33400
...,...,...,...,...
1242245,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-27,0
1242246,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-28,0
1242247,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-29,0
1242248,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-30,0


In [ ]:
estoque_regiao_mai_cmm = all_data_maio.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_mai_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-05,0.0
...,...,...,...,...
1242245,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-27,106848.0
1242246,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-28,106848.0
1242247,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-29,106848.0
1242248,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-30,106848.0


In [ ]:
estoque_regiao_mai = pd.merge(estoque_regiao_mai_estoque, estoque_regiao_mai_cmm, how = 'inner')

In [ ]:
estoque_regiao_mai = estoque_regiao_mai.replace(np.nan, 0)
estoque_regiao_mai

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-05,33400,0.0
...,...,...,...,...,...
1242245,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-27,0,106848.0
1242246,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-28,0,106848.0
1242247,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-29,0,106848.0
1242248,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-30,0,106848.0


Aqui começa o esquema estrela de modelagem para o mês de janeiro executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_mai): 
  id_supervisao = estoque_regiao_mai['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_mai['SUPERVISAO']= lista
estoque_regiao_mai = estoque_regiao_mai.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_mai

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-05,33400,0.0
...,...,...,...,...,...
1242245,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-27,0,106848.0
1242246,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-28,0,106848.0
1242247,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-29,0,106848.0
1242248,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-30,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_mai)): 
  id_descricao_item = estoque_regiao_mai['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_mai['DESCRICAO ITEM'] = lista
estoque_regiao_mai = estoque_regiao_mai.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_mai)

In [ ]:
estoque_regiao_mai.to_csv(r'estoque_regiao_mai.csv')

In [ ]:
estoque_regiao_mai

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-05-01,33400,0.0
1,1,61,2021-05-02,33400,0.0
2,1,61,2021-05-03,33400,0.0
3,1,61,2021-05-04,33400,0.0
4,1,61,2021-05-05,33400,0.0
...,...,...,...,...,...
1242245,32,1298,2021-05-27,0,106848.0
1242246,32,1298,2021-05-28,0,106848.0
1242247,32,1298,2021-05-29,0,106848.0
1242248,32,1298,2021-05-30,0,106848.0


#**Junho**

In [ ]:
all_data_junho = pd.DataFrame()
all_data_junho = all_data
all_data_junho

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,587,2021-06-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,470,2021-06-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,10,2021-06-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,13,2021-06-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,202,2021-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
453023,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-06-30
453024,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-06-30
453025,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-06-30
453026,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-06-30


In [ ]:
len(all_data_maio)

13925016

In [ ]:
all_data_junho["estoque"] = ""
all_data_junho["cmm_media"] = ""

In [ ]:
all_data_junho.to_csv(r'all_data_junho', index = False)

In [ ]:
estoque_regiao_jun_estoque = all_data_junho.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_jun_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-05,33400
...,...,...,...,...
1206303,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-26,0
1206304,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-27,0
1206305,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-28,0
1206306,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-29,0


In [ ]:
estoque_regiao_jun_cmm = all_data_junho.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_jun_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-05,0.0
...,...,...,...,...
1206303,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-26,106848.0
1206304,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-27,106848.0
1206305,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-28,106848.0
1206306,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-29,106848.0


In [ ]:
estoque_regiao_jun = pd.merge(estoque_regiao_jun_estoque, estoque_regiao_jun_cmm, how = 'inner')

In [ ]:
estoque_regiao_jun = estoque_regiao_jun.replace(np.nan, 0)
estoque_regiao_jun

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-05,33400,0.0
...,...,...,...,...,...
1206303,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-26,0,106848.0
1206304,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-27,0,106848.0
1206305,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-28,0,106848.0
1206306,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-29,0,106848.0


Aqui começa o esquema estrela de modelagem para o mês de janeiro executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_jun): 
  id_supervisao = estoque_regiao_jun['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_jun['SUPERVISAO']= lista
estoque_regiao_jun = estoque_regiao_jun.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_jun

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-05,33400,0.0
...,...,...,...,...,...
1206303,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-26,0,106848.0
1206304,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-27,0,106848.0
1206305,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-28,0,106848.0
1206306,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-29,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_jun)): 
  id_descricao_item = estoque_regiao_jun['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_jun['DESCRICAO ITEM'] = lista
estoque_regiao_jun = estoque_regiao_jun.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
estoque_regiao_jun.to_csv(r'estoque_regiao_jun.csv')

In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_jun)

#**Julho**

In [ ]:
all_data_julho = pd.DataFrame()
all_data_julho = all_data
all_data_julho

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,520,2021-07-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,430,2021-07-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,7,2021-07-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,8,2021-07-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,62,2021-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
459124,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-07-31
459125,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-07-31
459126,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-07-31
459127,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-07-31


In [ ]:
len(all_data_julho)

14165681

In [ ]:
all_data_julho["estoque"] = ""
all_data_julho["cmm_media"] = ""

In [ ]:
estoque_regiao_jul_estoque = all_data_julho.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_jul_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-05,33400
...,...,...,...,...
1270230,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-27,0
1270231,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-28,0
1270232,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-29,0
1270233,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-30,0


In [ ]:
estoque_regiao_jul_cmm = all_data_julho.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_jul_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-05,0.0
...,...,...,...,...
1270230,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-27,106848.0
1270231,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-28,106848.0
1270232,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-29,106848.0
1270233,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-30,106848.0


In [ ]:
estoque_regiao_jul = pd.merge(estoque_regiao_jul_estoque, estoque_regiao_jul_cmm, how = 'inner')

In [ ]:
estoque_regiao_jul = estoque_regiao_jul.replace(np.nan, 0)
estoque_regiao_jul

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-05,33400,0.0
...,...,...,...,...,...
1270230,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-27,0,106848.0
1270231,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-28,0,106848.0
1270232,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-29,0,106848.0
1270233,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-30,0,106848.0


Aqui começa o esquema estrela de modelagem para o mês de janeiro executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_jul): 
  id_supervisao = estoque_regiao_jul['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_jul['SUPERVISAO']= lista
estoque_regiao_jul = estoque_regiao_jul.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_jul

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-05,33400,0.0
...,...,...,...,...,...
1270230,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-27,0,106848.0
1270231,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-28,0,106848.0
1270232,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-29,0,106848.0
1270233,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-30,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_jul)): 
  id_descricao_item = estoque_regiao_jul['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_jul['DESCRICAO ITEM'] = lista
estoque_regiao_jul = estoque_regiao_jul.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
estoque_regiao_jul.to_csv(r'estoque_regiao_jul.csv')

In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_jul)

#**Agosto**

In [ ]:
all_data_agosto = pd.DataFrame()
all_data_agosto = all_data
all_data_agosto

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,467,2021-08-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,430,2021-08-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,107,2021-08-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5,2021-08-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,962,2021-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
467728,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-08-31
467729,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-08-31
467730,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-08-31
467731,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1,2021-08-31


In [ ]:
len(all_data_agosto)

13950753

In [ ]:
all_data_agosto["estoque"] = ""
all_data_agosto["cmm_media"] = ""

In [ ]:
estoque_regiao_ago_estoque = all_data_agosto.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_ago_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-04,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-05,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-06,33400
...,...,...,...,...
1246534,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-27,0
1246535,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-28,0
1246536,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-29,0
1246537,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-30,0


In [ ]:
estoque_regiao_ago_cmm = all_data_agosto.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_ago_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-04,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-05,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-06,0.0
...,...,...,...,...
1246534,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-27,106848.0
1246535,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-28,106848.0
1246536,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-29,106848.0
1246537,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-30,106848.0


In [ ]:
estoque_regiao_ago = pd.merge(estoque_regiao_ago_estoque, estoque_regiao_ago_cmm, how = 'inner')

In [ ]:
estoque_regiao_ago = estoque_regiao_ago.replace(np.nan, 0)
estoque_regiao_ago

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-04,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-05,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-06,33400,0.0
...,...,...,...,...,...
1246534,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-27,0,106848.0
1246535,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-28,0,106848.0
1246536,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-29,0,106848.0
1246537,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-30,0,106848.0


Aqui começa o esquema estrela de modelagem para o mês de janeiro executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_ago): 
  id_supervisao = estoque_regiao_ago['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_ago['SUPERVISAO']= lista
estoque_regiao_ago = estoque_regiao_ago.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_ago

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-04,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-05,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-06,33400,0.0
...,...,...,...,...,...
1246534,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-27,0,106848.0
1246535,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-28,0,106848.0
1246536,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-29,0,106848.0
1246537,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-30,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_ago)): 
  id_descricao_item = estoque_regiao_ago['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_ago['DESCRICAO ITEM'] = lista
estoque_regiao_ago = estoque_regiao_ago.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
estoque_regiao_ago.to_csv(r'estoque_regiao_ago.csv')

In [ ]:
estoque_regiao_ago

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-08-01,33400,0.0
1,1,61,2021-08-02,33400,0.0
2,1,61,2021-08-04,33400,0.0
3,1,61,2021-08-05,33400,0.0
4,1,61,2021-08-06,33400,0.0
...,...,...,...,...,...
1246534,32,1298,2021-08-27,0,106848.0
1246535,32,1298,2021-08-28,0,106848.0
1246536,32,1298,2021-08-29,0,106848.0
1246537,32,1298,2021-08-30,0,106848.0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_ago)

In [ ]:
estoque_estabelecimento_ago = all_data_agosto.groupby(['ESTABELECIMENTO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_estabelecimento_ago

ESTABELECIMENTO       DESCRICAO ITEM                                     data      
AE ALTO DA BOA VISTA  ABAIXADOR, LINGUA, MADEIRA, 100 UNIDADES           2021-08-01    4
                                                                         2021-08-02    4
                                                                         2021-08-04    4
                                                                         2021-08-05    4
                                                                         2021-08-06    4
                                                                                      ..
URSI TUCURUVI         VASOPRESSINA 20 UI/ML SOLUCAO INJETAVEL AMP. 1 ML  2021-08-27    2
                                                                         2021-08-28    2
                                                                         2021-08-29    2
                                                                         2021-08-30    2
                          

#**Setembro**

In [ ]:
all_data_setembro = pd.DataFrame()
all_data_setembro = all_data
all_data_setembro

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,581,2021-09-22
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,540,2021-09-22
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,103,2021-09-22
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5,2021-09-22
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,459,2021-09-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
469587,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-09-21
469588,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-09-21
469589,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-09-21
469590,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1,2021-09-21


In [ ]:
len(all_data_setembro)

14083854

In [ ]:
all_data_setembro["estoque"] = ""
all_data_setembro["cmm_media"] = ""

In [ ]:
estoque_regiao_set_estoque = all_data_setembro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_set_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-05,33400
...,...,...,...,...
1254239,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-26,0
1254240,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-27,0
1254241,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-28,0
1254242,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-29,0


In [ ]:
estoque_regiao_set_cmm = all_data_setembro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_set_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-05,0.0
...,...,...,...,...
1254239,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-26,106848.0
1254240,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-27,106848.0
1254241,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-28,106848.0
1254242,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-29,106848.0


In [ ]:
estoque_regiao_set = pd.merge(estoque_regiao_set_estoque, estoque_regiao_set_cmm, how = 'inner')

In [ ]:
estoque_regiao_set = estoque_regiao_set.replace(np.nan, 0)
estoque_regiao_set

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-05,33400,0.0
...,...,...,...,...,...
1254239,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-26,0,106848.0
1254240,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-27,0,106848.0
1254241,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-28,0,106848.0
1254242,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-29,0,106848.0


Aqui começa o esquema estrela de modelagem para o mês de janeiro executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_set): 
  id_supervisao = estoque_regiao_set['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_set['SUPERVISAO']= lista
estoque_regiao_set = estoque_regiao_set.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_set

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-05,33400,0.0
...,...,...,...,...,...
1254239,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-26,0,106848.0
1254240,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-27,0,106848.0
1254241,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-28,0,106848.0
1254242,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-29,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_set)): 
  id_descricao_item = estoque_regiao_set['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_set['DESCRICAO ITEM'] = lista
estoque_regiao_set = estoque_regiao_set.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
estoque_regiao_set.to_csv(r'estoque_regiao_set.csv')

In [ ]:
estoque_regiao_set

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-09-01,33400,0.0
1,1,61,2021-09-02,33400,0.0
2,1,61,2021-09-03,33400,0.0
3,1,61,2021-09-04,33400,0.0
4,1,61,2021-09-05,33400,0.0
...,...,...,...,...,...
1254239,32,1298,2021-09-26,0,106848.0
1254240,32,1298,2021-09-27,0,106848.0
1254241,32,1298,2021-09-28,0,106848.0
1254242,32,1298,2021-09-29,0,106848.0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_set)

#**Outubro**

In [ ]:
all_data_outubro = pd.DataFrame()
all_data_outubro = all_data
all_data_outubro

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,581,2021-10-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,530,2021-10-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,102,2021-10-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5,2021-10-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,179,2021-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486401,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-10-31
486402,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-10-31
486403,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-10-31
486404,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1,2021-10-31


In [ ]:
len(all_data_outubro)

14467024

In [ ]:
all_data_outubro["estoque"] = ""
all_data_outubro["cmm_media"] = ""

In [ ]:
estoque_regiao_out_estoque = all_data_outubro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_out_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-05,33400
...,...,...,...,...
1299871,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-27,0
1299872,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-28,0
1299873,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-29,0
1299874,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-30,0


In [ ]:
estoque_regiao_out_cmm = all_data_outubro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_out_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-05,0.0
...,...,...,...,...
1299871,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-27,106848.0
1299872,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-28,106848.0
1299873,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-29,106848.0
1299874,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-30,106848.0


In [ ]:
estoque_regiao_out = pd.merge(estoque_regiao_out_estoque, estoque_regiao_out_cmm, how = 'inner')

In [ ]:
estoque_regiao_out = estoque_regiao_out.replace(np.nan, 0)
estoque_regiao_out

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-05,33400,0.0
...,...,...,...,...,...
1299871,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-27,0,106848.0
1299872,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-28,0,106848.0
1299873,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-29,0,106848.0
1299874,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-30,0,106848.0


Aqui começa o esquema estrela de modelagem para o mês executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_out): 
  id_supervisao = estoque_regiao_out['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_out['SUPERVISAO']= lista
estoque_regiao_out = estoque_regiao_out.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_out

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-05,33400,0.0
...,...,...,...,...,...
1299871,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-27,0,106848.0
1299872,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-28,0,106848.0
1299873,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-29,0,106848.0
1299874,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-30,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_out)): 
  id_descricao_item = estoque_regiao_out['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_out['DESCRICAO ITEM'] = lista
estoque_regiao_out = estoque_regiao_out.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
estoque_regiao_out.to_csv(r'estoque_regiao_out.csv')

In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_out)

In [ ]:
estoque_regiao_out

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-10-01,33400,0.0
1,1,61,2021-10-02,33400,0.0
2,1,61,2021-10-03,33400,0.0
3,1,61,2021-10-04,33400,0.0
4,1,61,2021-10-05,33400,0.0
...,...,...,...,...,...
1299871,32,1298,2021-10-27,0,106848.0
1299872,32,1298,2021-10-28,0,106848.0
1299873,32,1298,2021-10-29,0,106848.0
1299874,32,1298,2021-10-30,0,106848.0


#**Novembro**

In [ ]:
all_data_novembro = pd.DataFrame()
all_data_novembro = all_data
all_data_novembro

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,579,2021-11-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,496,2021-11-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,100,2021-11-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5,2021-11-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,410,2021-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486618,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-11-17
486619,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-11-17
486620,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-11-17
486621,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1,2021-11-17


In [ ]:
len(all_data_novembro)

8270953

In [ ]:
all_data_novembro["estoque"] = ""
all_data_novembro["cmm_media"] = ""

In [ ]:
estoque_regiao_nov_estoque = all_data_novembro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum().reset_index()
estoque_regiao_nov_estoque

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-05,33400
...,...,...,...,...
743612,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-13,0
743613,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-14,0
743614,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-15,0
743615,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-16,0


In [ ]:
estoque_regiao_nov_cmm = all_data_novembro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['CMM'].mean().reset_index()
estoque_regiao_nov_cmm

,SUPERVISAO,DESCRICAO ITEM,data,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-01,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-02,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-03,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-04,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-05,0.0
...,...,...,...,...
743612,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-13,106848.0
743613,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-14,106848.0
743614,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-15,106848.0
743615,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-16,106848.0


In [ ]:
estoque_regiao_nov = pd.merge(estoque_regiao_nov_estoque, estoque_regiao_nov_cmm, how = 'inner')

In [ ]:
estoque_regiao_nov = estoque_regiao_nov.replace(np.nan, 0)
estoque_regiao_nov

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-05,33400,0.0
...,...,...,...,...,...
743612,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-13,0,106848.0
743613,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-14,0,106848.0
743614,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-15,0,106848.0
743615,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-16,0,106848.0


Aqui começa o esquema estrela de modelagem para o mês executado no ambiente:

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_nov): 
  id_supervisao = estoque_regiao_nov['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_nov['SUPERVISAO']= lista
estoque_regiao_nov = estoque_regiao_nov.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

In [ ]:
estoque_regiao_nov

,id_supervisao,DESCRICAO ITEM,data,ESTOQUE,CMM
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-01,33400,0.0
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-02,33400,0.0
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-03,33400,0.0
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-04,33400,0.0
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-05,33400,0.0
...,...,...,...,...,...
743612,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-13,0,106848.0
743613,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-14,0,106848.0
743614,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-15,0,106848.0
743615,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-16,0,106848.0


descrição para fato!

In [ ]:
n = 0
lista = []
while n < (len(estoque_regiao_nov)): 
  id_descricao_item = estoque_regiao_nov['DESCRICAO ITEM'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
estoque_regiao_nov['DESCRICAO ITEM'] = lista
estoque_regiao_nov = estoque_regiao_nov.rename(columns = {'DESCRICAO ITEM': 'id_descricao_item'}, inplace = False)

In [ ]:
estoque_regiao_nov.to_csv(r'estoque_regiao_nov.csv')

In [ ]:
estoque_regiao_nov

,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,1,61,2021-11-01,33400,0.0
1,1,61,2021-11-02,33400,0.0
2,1,61,2021-11-03,33400,0.0
3,1,61,2021-11-04,33400,0.0
4,1,61,2021-11-05,33400,0.0
...,...,...,...,...,...
743612,32,1298,2021-11-13,0,106848.0
743613,32,1298,2021-11-14,0,106848.0
743614,32,1298,2021-11-15,0,106848.0
743615,32,1298,2021-11-16,0,106848.0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_nov)

#**Estoque região - único arquivo**

In [ ]:
remedios_demanda_regiao = pd.DataFrame()

In [ ]:
remedios_demanda_regiao

""


#**Salvando**

In [ ]:
remedios_demanda_regiao

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,33400.0,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,33400.0,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,33400.0,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,33400.0,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,33400.0,0.0
...,...,...,...,...,...
1236132,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-27,0.0,106848.0
1236133,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-28,0.0,106848.0
1236134,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-29,0.0,106848.0
1236135,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-30,0.0,106848.0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.rename(columns={'SUPERVISAO':'supervisao','DESCRICAO ITEM':'descricao_item','ESTOQUE':'estoque'})

In [ ]:
remedios_demanda_regiao.to_csv(r'/remedios_estoque_regiao_fato.csv', index = False)

In [ ]:
remedios_demanda_regiao

,id_supervisao,id_descricao_item,data,estoque
0,1,61,2021-01-01,33400
1,1,61,2021-01-02,33400
2,1,61,2021-01-03,33400
3,1,61,2021-01-04,33400
4,1,61,2021-01-05,33400
...,...,...,...,...
13055609,32,1298,2021-11-13,0
13055610,32,1298,2021-11-14,0
13055611,32,1298,2021-11-15,0
13055612,32,1298,2021-11-16,0


#**remedios_estoque_regiao_dim_supervisao**

Duas opções, refazer a dim ou ler a feita:

In [ ]:
estoque_regiao_dim_supervisao = pd.DataFrame()
estoque_regiao_dim_supervisao['supervisao'] = remedios_demanda_regiao['supervisao'].drop_duplicates()
estoque_regiao_dim_supervisao = estoque_regiao_dim_supervisao.sort_values(by=['supervisao'], ascending=True)
estoque_regiao_dim_supervisao.index = np.arange(1, len(estoque_regiao_dim_supervisao) + 1)
estoque_regiao_dim_supervisao['id_supervisao'] = estoque_regiao_dim_supervisao.index

In [ ]:
estoque_regiao_dim_supervisao = pd.read_csv('remedios_estoque_regiao_dim_supervisao.csv')

In [ ]:
estoque_regiao_dim_supervisao

,supervisao,id_supervisao
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,1
1,SECRETARIA MUNICIPAL DA SAUDE,2
2,SUPERVISAO TECNICA DE SAUDE ERMELINO MATARAZZO,3
3,SUPERVISAO TECNICA DE SAUDE HOSPITALAR,4
4,SUPERVISAO TECNICA DE SAUDE PERUS,5
5,SUPERVISAO TECNICA DE SAUDE SANTA CECILIA,6
6,SUPERVISAO TECNICA DE SAUDE SAO MIGUEL,7
7,SUPERVISÃO TÉCNICA DE SAÚDE ARICANDUVA/FORMOSA...,8
8,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,9
9,SUPERVISÃO TÉCNICA DE SAÚDE CAMPO LIMPO,10


In [ ]:
estoque_regiao_dim_supervisao.to_csv(r'remedios_estoque_regiao_dim_supervisao.csv', index = False)

In [ ]:
def ConverteSupervisaoNumID(id_supervisao):
    dicionario_supervisao = dict(sorted(estoque_regiao_dim_supervisao.values.tolist())) 
    return dicionario_supervisao[id_supervisao]

In [ ]:
estoque_regiao_jan

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE,CMM
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,33400,0.0
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,33400,0.0
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,33400,0.0
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,33400,0.0
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,33400,0.0
...,...,...,...,...,...
1236132,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-27,0,106848.0
1236133,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-28,0,106848.0
1236134,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-29,0,106848.0
1236135,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-30,0,106848.0


In [ ]:
len(estoque_regiao_jan)

1236137

In [ ]:
n = 0
lista = []
while n < len(estoque_regiao_jan): 
  id_supervisao = estoque_regiao_jan['SUPERVISAO'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
estoque_regiao_jan['SUPERVISAO']= lista
estoque_regiao_jan = estoque_regiao_jan.rename(columns = {'SUPERVISAO': 'id_supervisao'}, inplace = False)

#**remedios_estoque_regiao_dim_descricao_item**

In [ ]:
estoque_regiao_dim_descricao_item = pd.DataFrame()
estoque_regiao_dim_descricao_item['descricao_item'] = remedios_demanda_regiao['descricao_item'].drop_duplicates()
estoque_regiao_dim_descricao_item = estoque_regiao_dim_descricao_item.sort_values(by=['descricao_item'], ascending=True)
estoque_regiao_dim_descricao_item.index = np.arange(1, len(estoque_regiao_dim_descricao_item) + 1)
estoque_regiao_dim_descricao_item['id_descricao_item'] = estoque_regiao_dim_descricao_item.index

In [ ]:
estoque_regiao_dim_descricao_item = pd.read_csv('remedios_estoque_regiao_dim_descricao_item.csv')

In [ ]:
estoque_regiao_dim_descricao_item

,descricao_item,id_descricao_item
0,-- NAO UTILIZAR -- ALENDRONATO DE SODIO EM COM...,1
1,-- NAO UTILIZAR -- ASSOCIACAO DE LEVODOPA 200 ...,2
2,-- NAO UTILIZAR -- LEVODOPA 100 MG + CLORIDRAT...,3
3,-- NAO UTILIZAR -- LEVODOPA 200 MG + CLORIDRAT...,4
4,-- NAO UTILIZAR -- SINVASTATINA EM COMPRIMIDOS...,5
...,...,...
3278,WATER TRAP 1 ADULTO/PEDIATRICO,3279
3279,"XILOL, P.A.",3280
3280,ZANAMIVIR 5 MG PO PARA SOLUCAO INALANTE,3281
3281,ZIDOVUDINA (AZT) 10 MG/ML SOLUCAO ORAL FRASCO ...,3282


In [ ]:
estoque_regiao_dim_descricao_item.to_csv(r'remedios_estoque_regiao_dim_descricao_item.csv', index = False)

In [ ]:
def Converte_descricao_item_NumID(id_descricao_item):
    dicionario_descricao_item = dict(sorted(estoque_regiao_dim_descricao_item.values.tolist())) 
    return dicionario_descricao_item[id_descricao_item]

In [ ]:
n = 0
lista = []
while n < (len(remedios_demanda_regiao)/10): 
  id_descricao_item = remedios_demanda_regiao['descricao_item'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
n = (int(len(remedios_demanda_regiao))/10)*1
lista = []
while n < (len(remedios_demanda_regiao)/10)*2: 
  id_descricao_item = remedios_demanda_regiao['descricao_item'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
n = (len(remedios_demanda_regiao)/10)*2
lista = []
while n < (len(remedios_demanda_regiao)/10)*3: 
  id_descricao_item = remedios_demanda_regiao['descricao_item'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
remedios_demanda_regiao['descricao_item']= lista
remedios_demanda_regiao = remedios_demanda_regiao.rename(columns = {'descricao_item': 'id_descricao_item'}, inplace = False)

In [ ]:
remedios_demanda_regiao.to_csv(r'remedios_estoque_regiao_fato_incomplete_1.csv', index = False)

#**Juntando os DF gerados para cada mês:**

In [20]:
modelados = glob.glob('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/*.csv')
modelados

['/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/06-estoque_regiao_jun.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/09-estoque_regiao_set.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/10-estoque_regiao_out.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/11-estoque_regiao_nov.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/03-estoque_regiao_mar.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/05-estoque_regiao_mai.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/02-estoque_regiao_fev.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/01-estoque_regiao_jan.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/modelados_mes/04-estoque_regiao_abr.csv',
 '/content/drive/My Drive/Prefeitura/

In [7]:
all_data = pd.DataFrame() 
n = 0
for f in modelados:
    df1 = pd.read_csv(f)
    all_data = all_data.append(df1)
    n += 1
all_data

,Unnamed: 0,id_supervisao,id_descricao_item,data,ESTOQUE,CMM
0,0,1,61,2021-06-01,33400,0.0
1,1,1,61,2021-06-02,33400,0.0
2,2,1,61,2021-06-03,33400,0.0
3,3,1,61,2021-06-04,33400,0.0
4,4,1,61,2021-06-05,33400,0.0
...,...,...,...,...,...,...
1246534,1246534,32,1298,2021-08-27,0,106848.0
1246535,1246535,32,1298,2021-08-28,0,106848.0
1246536,1246536,32,1298,2021-08-29,0,106848.0
1246537,1246537,32,1298,2021-08-30,0,106848.0


In [34]:
all_data = all_data.rename(columns={'ESTOQUE':'estoque','CMM':'cmm'})

In [ ]:
all_data = all_data.drop(columns=['Unnamed: 0'])

In [39]:
all_data.to_csv(r'medicamento_historico_fato_suprimentos.csv', index = False)

In [45]:
all_data

,id_supervisao,id_descricao_item,data,estoque,cmm
0,1,61,2021-01-01,33400,0.0
1,1,61,2021-01-02,33400,0.0
2,1,61,2021-01-03,33400,0.0
3,1,61,2021-01-04,33400,0.0
4,1,61,2021-01-05,33400,0.0
...,...,...,...,...,...
743612,32,1298,2021-11-13,0,106848.0
743613,32,1298,2021-11-14,0,106848.0
743614,32,1298,2021-11-15,0,106848.0
743615,32,1298,2021-11-16,0,106848.0


In [11]:
all_data.data.unique() #verificando se todas as datas estão inclusas como deve, pois o arquivo ficou menor do que o esperado

array(['2021-06-01', '2021-06-02', '2021-06-03', '2021-06-04',
       '2021-06-05', '2021-06-06', '2021-06-07', '2021-06-08',
       '2021-06-09', '2021-06-10', '2021-06-11', '2021-06-12',
       '2021-06-13', '2021-06-14', '2021-06-15', '2021-06-16',
       '2021-06-17', '2021-06-18', '2021-06-19', '2021-06-20',
       '2021-06-21', '2021-06-22', '2021-06-23', '2021-06-24',
       '2021-06-25', '2021-06-26', '2021-06-27', '2021-06-28',
       '2021-06-29', '2021-06-30', '2021-09-01', '2021-09-02',
       '2021-09-03', '2021-09-04', '2021-09-05', '2021-09-06',
       '2021-09-07', '2021-09-08', '2021-09-09', '2021-09-10',
       '2021-09-11', '2021-09-12', '2021-09-13', '2021-09-14',
       '2021-09-15', '2021-09-16', '2021-09-17', '2021-09-18',
       '2021-09-19', '2021-09-20', '2021-09-21', '2021-09-22',
       '2021-09-23', '2021-09-24', '2021-09-25', '2021-09-26',
       '2021-09-27', '2021-09-28', '2021-09-29', '2021-09-30',
       '2021-10-01', '2021-10-02', '2021-10-03', '2021-

#Provas reais

In [ ]:
data_n = extraidos[0].split('/')
data_n = data_n[9].split('_')
data_n = data_n[3].split('.')
data_n = int(data_n[0])
data_n

20210101

In [ ]:
data_n_ano = data_n // 10000
data_n_ano

2021

In [ ]:
data_n_mes = (data_n % 10000)//100
data_n_mes 

9

In [ ]:
data_n_dia = data_n % 10000
if data_n_dia > 999:
  data_n_dia = data_n_dia%100
else:
  data_n_dia = data_n_dia%100
data_n_dia

23

In [ ]:
date(year=data_n_ano, month=data_n_mes, day=data_n_dia)

datetime.date(2021, 9, 23)

In [ ]:
for f in extraidos:
  print(f)

/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210923.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210924.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210925.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210926.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210927.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210928.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210929.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210930.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20211001.csv
/content/drive/My Drive/Prefeitura/Dados/situa

In [ ]:
for f in extraidos:
  data_n = extraidos[f].split('/')
  data_n = data_n[8].split('_')
  data_n = data_n[3].split('.')
  data_n = data_n[0]
  data_n = str(data_n)
  data_n = int(data_n)
  print(data_n)

TypeError: ignored

#lab de operações!



In [ ]:
#Se houvesse vários arquivos esse codigo era uma boa!
#for n in arquivos:
#    for file in arquivos[0].namelist():
#        arquivos.extract(file, '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/')

In [ ]:
with zipfile.ZipFile('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/Novembro/gss_posicao_estoque_20211102.zip') as zip:
    lista = z.namelist()

In [ ]:
lista

['gss_posicao_estoque_20211101.csv']

In [ ]:
with zipfile.ZipFile('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/Novembro/gss_posicao_estoque_20211102.zip') as zip:
  with z.open('gss_posicao_estoque_20211101.csv') as f:
    df = pd.read_csv(f, sep = ';', header = 0, encoding='ISO-8859-1', nrows=0 )

ValueError: ignored

In [ ]:
print(df)

                                                    0
0   /content/drive/My Drive/Prefeitura/Dados/situa...
1   /content/drive/My Drive/Prefeitura/Dados/situa...
2   /content/drive/My Drive/Prefeitura/Dados/situa...
3   /content/drive/My Drive/Prefeitura/Dados/situa...
4   /content/drive/My Drive/Prefeitura/Dados/situa...
5   /content/drive/My Drive/Prefeitura/Dados/situa...
6   /content/drive/My Drive/Prefeitura/Dados/situa...
7   /content/drive/My Drive/Prefeitura/Dados/situa...
8   /content/drive/My Drive/Prefeitura/Dados/situa...
9   /content/drive/My Drive/Prefeitura/Dados/situa...
10  /content/drive/My Drive/Prefeitura/Dados/situa...
11  /content/drive/My Drive/Prefeitura/Dados/situa...
12  /content/drive/My Drive/Prefeitura/Dados/situa...
13  /content/drive/My Drive/Prefeitura/Dados/situa...
14  /content/drive/My Drive/Prefeitura/Dados/situa...
15  /content/drive/My Drive/Prefeitura/Dados/situa...
16  /content/drive/My Drive/Prefeitura/Dados/situa...


In [ ]:
data_frame = pd.DataFrame()
for n in arquivos:
    with zipfile.ZipFile(n) as z:
        with z.open(n) as f:
            dados = pd.read_csv(f, sep = ';', header = 0, encoding='ISO-8859-1',nrows=0)
            all_data = pd.concat(dados, axis = 1)